# 파이썬과 Numpy를 이용한 산술연산

![](https://i.imgur.com/mg8O3kd.png)


본 장에서는 다음과 같은 주제를 다룹니다:

___1. 파이썬에서 수치 데이터 다루기___<br>
___2. 파이썬 리스트에서 넘파이 배열(Numpy Array)로___<br>
___3. 다차원 넘파이 배열과 장점의 소개___<br>
___4. 배열 연산, 브로드캐스팅(broadcasting), 인덱싱(indexing), 슬라이싱(slicing)___<br>
___5. 넘파이를 사용해서 CSV 파일 다루기___<br>

## 1. 파이썬에서 수치 데이터 다루기

데이터 분석(Data Analysis)에서 '데이터(Data)'는 보통 수치 자료를 다룬다.<br>
ex) 주가, 판매량, 센서 값, 스포츠 점수, 데이터베이스 표, etc.<br>
[Numpy](https://numpy.org) 라이브러리는 파이썬내에서 수리 계산에 유용한 __데이터 구조, 함수, 다른 기능들__을 제공한다.<br>
이제 예시들을 보면서 넘파이가 어떻게 수치 데이터 연산에 유용하게 쓰이는지 보도록하자.


>당신이 사과나무를 심고자하는 농부라고 생각해보자. 이제 당신은 기온, 강수량, 습도 등과 같은 기후 데이터를 이용해서 어떤 지역이 사과나무를 심기 가장 적절한지 고민해야한다. 고민을 해결할 수 있는 하나의 방법은 아래와 같이 기후 데이터들과 연간 사과 수확량 사이의 관계를 보이는 단순한 식을 만들어 내는 것이다.
> `yield_of_apples = w1 * temperature + w2 * rainfall + w3 * humidity`

우리는 연간 사과 수확량을 기온, 강수량, 습도와의 가중평균을 이용하여 표현하였다.<br> 
실제로는 관계가 선형적이지 아닐 수도 있고, 수확량에 영향을 주는 다른 변수들이 존재할 수도 있다.<br> 
그러나 단순 선형회귀는 좋은 결과를 만들어 내기도 한다.<br>

과거 데이터들에 대한 몇가지 통계 처리를 거친다면 각 변수들의 가중치인  `w1`, `w2`, `w3` 를 구할 수 있을 것이다.<br> 
예시를 보자:

In [2]:
w1, w2, w3 = 0.3, 0.2, 0.5

지역에 대한 기후 데이터가 주어지면, 이제 사과 수확량을 예측할 수 있다.<br>
예시 데이터는 아래와 같다:

<img src="https://i.imgur.com/TXPBiqv.png" style="width:360px;">

시작하기 앞서, 데이터 이름을 정해야한다.

In [3]:
kanto_temp = 73
kanto_rainfall = 67
kanto_humidity = 43

이제 우리는 이 변수들을 사과 수확량을 예측하는 식에 대입할 수 있다.

In [4]:
kanto_yield_apples = kanto_temp * w1 + kanto_rainfall * w2 + kanto_humidity * w3
kanto_yield_apples

56.8

In [5]:
print(f"The expected yield of apples in Kanto region is {kanto_yield_apples} tons per hectare.")

The expected yield of apples in Kanto region is 56.8 tons per hectare.


위 식을 다중 선형 회귀(multiple liner regression)으로 만들기 위해서,<br>
각 지역의 기후데이터를 벡터 형태로 만드는 것이 좋다.<br>

In [6]:
kanto = [73, 67, 43]
johto = [91, 88, 64]
hoenn = [87, 134, 58]
sinnoh = [102, 43, 37]
unova = [69, 96, 70]

벡터의 세가지 값들은 각각  기온, 강우량, 습도 데이터를 의미한다.<br><br>
그리고 가중치도 벡터 변수 형태로 만들어 사용한다.

In [7]:
weights = [w1, w2, w3]

이제 함수 `crop_yield`를 만들어 주어진 지역별 기후데이터와 가중치 데이터를 사용해서 사과 수확량을 예측해보도록 한다. 

In [8]:
def crop_yield(region, weights):
    result = 0
    for x, w in zip(region, weights):
        result += x * w
    return result

In [9]:
crop_yield(kanto, weights)

56.8

In [10]:
crop_yield(johto, weights)

76.9

In [11]:
crop_yield(unova, weights)

74.9

## 2.1 파이썬 리스트에서 넘파이 배열로


함수 `crop_yield` 를 이용해서 수행된 계산은 (두 벡터의 요소별 곱셈 및 결과의 합) <font color=red>__내적(dot product)__</font> 이라고 불린다.<br>
내적 연산엦 관한 내용은 다음 웹페이지에서 설명하고 있다: https://www.khanacademy.org/math/linear-algebra/vectors-and-spaces/dot-cross-products/v/vector-dot-product-and-vector-length . 

넘파이 라이브러리는 __내적을 계산해주는 함수__를 제공해준다.<br>
그러나, 우리는 일단 이 파이썬 리스트 형태의 데이터를 넘파이 배열 형태로 바꾸는 작업부터 해주어야한다.

__'pip'__ 을 이용하여 넘파이를 설치하는 방법을 알아보자

In [12]:
#!pip install numpy --upgrade --quiet

Numpy 설치가 완료되었다!<br>
이제 `numpy` 를 임포트 하여 사용할 수 있다.<br>
`numpy`는 보통 `np`라는 이름으로 사용한다.

In [13]:
import numpy as np

이제 `np.array` 함수를 이용해서 넘파이 배열을 만들 수 있다.

In [14]:
kanto = np.array([73, 67, 43])

In [15]:
kanto

array([73, 67, 43])

In [16]:
weights = np.array([w1, w2, w3])

In [17]:
weights

array([0.3, 0.2, 0.5])

넘파이 배열은 __`ndarray` 자료형__으로 만들어져 있다.

In [18]:
type(kanto)

numpy.ndarray

In [19]:
type(weights)

numpy.ndarray

리스트와 같이, 넘파이 배열도 __`[]`__ 를 이용하여 __인덱스__로 접근할 수 있다.

In [20]:
weights[0]

0.3

In [21]:
kanto[2]

43

## 2.2 넘파이 배열의 연산

이제 우리는 넘파이의 __`np.dot()`함수__ 를 이용하여 내적(fot product)연산을 쉽게 할 수 있다.

In [22]:
np.dot(kanto, weights)

56.8

같은 연산 결과를 보다 낮은 수준의 연산자들을 복합적으로 이용하여 구할 수도 있다:<br> 
원소별 곱셈을 한 후에 해당 결과들의 합을 구하는 방식이다.

In [23]:
(kanto * weights).sum()

56.8

 `*` 는 원소별 곱셈을 수행해주는 연산자이다 <font color=red>(두 넘파이 배열의 크기가 같은 경우에만 가능하다)</font><br>
 `sum`은 배열내 수치 값들의 합을 계산해주는 연산자이다.

In [24]:
arr1 = np.array([1, 2, 3])
arr2 = np.array([4, 5, 6])

In [25]:
arr1 * arr2

array([ 4, 10, 18])

In [26]:
arr2.sum()

15

## 3. 넘파이 배열의 장점

넘파이 배열은 파이썬 리스트에 비해 다음과 같은 장점을 가지고 있다:
<font color = blue>
- **손쉬운 사용**: `crop_yield`와 같이 사용자가 정의 함수를 사용하는 것 대신,  `(kanto * weights).sum()`와 같이 간결하고 직관적인 수학 표현식을 작성할 수 있다.
- **성능**: 넘파이 연산은 내부적으로 C++로 작동된다. 따라서 파이썬의 기본 연산들보다 빠른 속도로 연산이 가능해진다.
</font>

파이썬과 넘파이 배열의 성능 차이를 보여주는 몇가지 예시 문제를 보도록 한다.

In [27]:
# 파이썬 리스트
arr1 = list(range(1000000))
arr2 = list(range(1000000, 2000000))

# 너ㅗㅁ
arr1_np = np.array(arr1)
arr2_np = np.array(arr2)

In [28]:
%%time
result = 0
for x1, x2 in zip(arr1, arr2):
    result += x1*x2
result

Wall time: 167 ms


833332333333500000

In [29]:
%%time
np.dot(arr1_np, arr2_np)

Wall time: 997 µs


-1942957984

넘파이의 `np.dot` 를 사용하는 것이 `for` 반복문을 사용하는 것보다 100배 정도 빠르다는 것을 알 수 있었다.<br>
이러한 특징은 대량의 데이터와 복잡한 연산구조의 모델이 작동할 때 파이썬 리스트보다 넘파이가 유용하게 만들어 준다.

In [30]:
import jovian

ModuleNotFoundError: No module named 'jovian'

In [31]:
jovian.commit()

NameError: name 'jovian' is not defined

---
## 3.2 다차원 넘파이 배열 

이제 한발짝 더 나아가서, 지역의 데이터들을 2차원의 넘파이 배열로 만들어 보도록 하자

In [32]:
climate_data = np.array([[73, 67, 43],
                         [91, 88, 64],
                         [87, 134, 58],
                         [102, 43, 37],
                         [69, 96, 70]])

In [33]:
climate_data

array([[ 73,  67,  43],
       [ 91,  88,  64],
       [ 87, 134,  58],
       [102,  43,  37],
       [ 69,  96,  70]])

위의 2차원 배열은 5개의 행(row)과 3개의 열(colums)를 가지고 있다.<br>
각 열들은 하나의 지역을 의미하고 행들은 각각 기온, 강수량, 습도를 의미한다.<br><br>

넘파이 배열은 어떠한 차원이라도 표현이가능하고 다양한 형태가 존재한다. <br>
넘파이 배열의 `.shape` 특성을 이용한다면 해당 배열의 차원을 알 수 있다. 

<img src="https://fgnt.github.io/python_crashkurs_doc/_images/numpy_array_t.png" width="420">



In [37]:
# 2차원 배열(행렬: Matrix)
climate_data.shape

(5, 3)

In [38]:
weights

array([0.3, 0.2, 0.5])

In [39]:
# 1차원 배열(벡터 : Vector)
weights.shape

(3,)

In [40]:
# 3차원 배열 
arr3 = np.array([
    [[11, 12, 13], 
     [13, 14, 15]], 
    [[15, 16, 17], 
     [17, 18, 19.5]]])

In [41]:
arr3.shape

(2, 2, 3)

단일 넘파이 배열 내의 모든 원소들은 __<font color=red> 하나의 자료형으로만 존재할 수 있다</font>__. <br>
`.dtype` 특성을 이용한다면 원소들의 자료형을 알 수 있다.

In [42]:
weights.dtype

dtype('float64')

In [43]:
climate_data.dtype

dtype('int32')

배열내에 하나의 원소만 실수형 일지라도<br> 
모든 다른 원소들은 실수형으로 변환된다.

In [44]:
arr3.dtype

dtype('float64')

### 행렬곱 연산 : np.matmul(), @
우리는 이제 한번의 행렬곱만으로도 사과 수확량을 예측할 수 있게 되었다.<br>
행렬곱은 5x3 행렬로 기후 데이터를 가지고 있는 `climate_data`와 <br>
3x1 배열로 가중치를 가지고 있는 `weights`사이에서 이루어 진다.<br>
과정은 아래와 같이 시각화 될 수 있다.

<img src="https://i.imgur.com/LJ2WKSI.png" width="240">

행렬곱 연산에 대해 보다 자세하게 알고 싶다면 옆의 동영상 플레이리스트의 3번영상 까지를 참고하는 것도 좋다: https://www.youtube.com/watch?v=xyAuNHPsq-g&list=PLFD0EB975BA0CC1E0&index=1 .

__`np.matmul` 함수__ 나 __`@` 연산자__ 는 넘파이 배열 사이의 행렬곱 연산을 쉽게 해준다.

In [45]:
np.matmul(climate_data, weights)

array([56.8, 76.9, 81.9, 57.7, 74.9])

In [46]:
climate_data @ weights

array([56.8, 76.9, 81.9, 57.7, 74.9])

---
## 4. 넘파이를 이용한 CSV 데이터 다루기

넘파이는 __데이터를 읽고 쓰는 것을 지원하는 함수__ 들을 제공하고 있다.<br>
만개의 기후 측정 데이터를 가지고 있는 `climate.txt`파일을 다운로드 받아보도록 한다.

```
temperature,rainfall,humidity
25.00,76.00,99.00
39.00,65.00,70.00
59.00,45.00,77.00
84.00,63.00,38.00
66.00,50.00,52.00
41.00,94.00,77.00
91.00,57.00,96.00
49.00,96.00,99.00
67.00,20.00,28.00
...
```

이러한 데이터 저장 형태를  ___쉼표로 구분된 파일___ 또는 __CSV__ 라고 한다. 

> **CSVs**: A comma-separated values (CSV) file is a delimited text file that uses a comma to separate values. Each line of the file is a data record. Each record consists of one or more fields, separated by commas. A CSV file typically stores tabular data (numbers and text) in plain text, in which case each line will have the same number of fields. (Wikipedia)


이러한 파일을 넘파이 배열로 읽어 내기 위해서 `genfromtxt` 함수를 사용한다.

In [47]:
import urllib.request

urllib.request.urlretrieve(
    'https://hub.jovian.ml/wp-content/uploads/2020/08/climate.csv', 
    'climate.txt')

('climate.txt', <http.client.HTTPMessage at 0x2ef7b536e50>)

In [48]:
climate_data = np.genfromtxt('climate.txt', delimiter=',', skip_header=1)

In [49]:
climate_data

array([[25., 76., 99.],
       [39., 65., 70.],
       [59., 45., 77.],
       ...,
       [99., 62., 58.],
       [70., 71., 91.],
       [92., 39., 76.]])

In [50]:
climate_data.shape

(10000, 3)

우리는 이제 `@` 연산자를 이용해서 전체 데이터 셋에 대한 가중치 계산을 쉽게 해낼 수 있다.

In [51]:
weights = np.array([0.3, 0.2, 0.5])

In [52]:
yields = climate_data @ weights

In [53]:
yields

array([72.2, 59.7, 65.2, ..., 71.1, 80.7, 73.4])

In [54]:
yields.shape

(10000,)

 `yields` 를 `climate_data` 의 네번째 열로 추가해보도록 한다.<br> 
 [`np.concatenate`](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html) 함수를 이용하면 이와 같은 연산을 쉽게 할 수 있다.

In [55]:
climate_results = np.concatenate((climate_data, yields.reshape(10000, 1)), axis=1)

In [56]:
climate_results

array([[25. , 76. , 99. , 72.2],
       [39. , 65. , 70. , 59.7],
       [59. , 45. , 77. , 65.2],
       ...,
       [99. , 62. , 58. , 71.1],
       [70. , 71. , 91. , 80.7],
       [92. , 39. , 76. , 73.4]])


* 새로운 열을 추가하고 싶을 때는, `np.concatenate`함수에 새로운 파라미터 `axis=1` 를 추가하게 된다.`axis`는 배열을 합치는 방향(차원)을 지정해 주는 기능을 한다.

*  배열을 합칠 때는 합치는 대상이 되는 차원 이외의 다른 차원들은 같은 길이와 깊이를 가지고 있어야한다. <br> We use the [`np.reshape`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) function to change the shape of `yields` from `(10000,)` to `(10000,1)`.

Here's a visual explanation of `np.concatenate` along `axis=1` (can you guess what `axis=0` results in?):

<img src="https://www.w3resource.com/w3r_images/python-numpy-image-exercise-58.png" width="300">

The best way to understand what a Numpy function does is to experiment with it and read the documentation to learn about its arguments & return values. Use the cells below to experiment with `np.concatenate` and `np.reshape`.

최종 결과물이 담겨진 넘파이 배열을 `np.savetxt` 함수를 이용하여 파일로 저장해보자.

In [57]:
climate_results

array([[25. , 76. , 99. , 72.2],
       [39. , 65. , 70. , 59.7],
       [59. , 45. , 77. , 65.2],
       ...,
       [99. , 62. , 58. , 71.1],
       [70. , 71. , 91. , 80.7],
       [92. , 39. , 76. , 73.4]])

In [58]:
np.savetxt('climate_results.txt', 
           climate_results, 
           fmt='%.2f', 
           delimiter=',',
           header='temperature,rainfall,humidity,yeild_apples', 
           comments='')

결과물은 `climate_results.txt` 라는 이름을 가진 CSV 형태의 파일로 저장되게 된다|. 

```
temperature,rainfall,humidity,yeild_apples
25.00,76.00,99.00,72.20
39.00,65.00,70.00,59.70
59.00,45.00,77.00,65.20
84.00,63.00,38.00,56.80
...
```



---
넘파이는 배열 연산을 위한 수백가지 함수들을 제공한다:


* 수리연산: `np.sum`, `np.exp`, `np.round`, 산술 연산자들(+, -, *, /)
* 배열 생성: `np.reshape`, `np.stack`, `np.concatenate`, `np.split`
* 선형 대수: `np.matmul`, `np.dot`, `np.transpose`, `np.eigvals`
* 통계: `np.mean`, `np.median`, `np.std`, `np.max`

> **필요한 함수들은 어떻게 찾을 수 있나요?**<br>
필요한 기능을 하는 함수들을 찾는 가장 쉬운 방법은 인터넷에 검색하는 것입니다. 예를들어 "어떻게 넘파이 배열을 합칠 수 있을까요?"라는 질문은 이 페이지로 이어집니다. [this tutorial on array concatenation](https://cmdlinetips.com/2018/04/how-to-concatenate-arrays-in-numpy/). 

넘파이 배열과 관련한 모든 함수들은 이곳에 명시되어있습니다: https://numpy.org/doc/stable/reference/routines.html

## 5.1 배열연산, 브로드캐스팅, 비교연산

넘파이 배열은 수리 연산자 `+`, `-`, `*`등을 지원한다.<br>
스칼라를 비롯한 다양한 넘파이 배열에 대해 연산을 수행할 수 있다.<br>
연산자는 다차원 배열들에 대한 수학적 표현을 쉽게 만들어 준다.

In [59]:
arr2 = np.array([[1, 2, 3, 4], 
                 [5, 6, 7, 8], 
                 [9, 1, 2, 3]])

In [60]:
arr3 = np.array([[11, 12, 13, 14], 
                 [15, 16, 17, 18], 
                 [19, 11, 12, 13]])

In [61]:
# 스칼라 더하기
arr2 + 3

array([[ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12,  4,  5,  6]])

In [62]:
# 원소별 빼기
arr3 - arr2

array([[10, 10, 10, 10],
       [10, 10, 10, 10],
       [10, 10, 10, 10]])

In [63]:
# 스칼라 값으로 나누기
arr2 / 2

array([[0.5, 1. , 1.5, 2. ],
       [2.5, 3. , 3.5, 4. ],
       [4.5, 0.5, 1. , 1.5]])

In [64]:
# 원소별 곱하기
arr2 * arr3

array([[ 11,  24,  39,  56],
       [ 75,  96, 119, 144],
       [171,  11,  24,  39]])

In [65]:
# 스칼라 나머지 연산하기
arr2 % 4

array([[1, 2, 3, 0],
       [1, 2, 3, 0],
       [1, 1, 2, 3]], dtype=int32)

### 배열 브로드캐스팅

넘파이 배열은 __브로드캐스팅(broad casting)__ 기능을 지원한다<br>
브로드 캐스팅은 차원이 다른 두 배열 간의 연산이 가능하도록 지원한다.

In [66]:
arr2 = np.array([[1, 2, 3, 4], 
                 [5, 6, 7, 8], 
                 [9, 1, 2, 3]])

In [67]:
arr2.shape

(3, 4)

In [68]:
arr4 = np.array([4, 5, 6, 7])

In [69]:
arr4.shape

(4,)

In [70]:
arr2 + arr4

array([[ 5,  7,  9, 11],
       [ 9, 11, 13, 15],
       [13,  6,  8, 10]])

`arr2 + arr4` 라는 식이 계산될 때, `arr4` 는 `arr2`의 형태인  `(3, 4)`과 연산하기 위해 모양을 맞춘다.<br>
넘파이는 실제로 배열의 크기를 변경하는 것 없이 자동으로 연산이 가능하도록 연산식을 수정해준다.<br>

<img src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/02.05-broadcasting.png" width="360">

브로드캐스팅은 두 연산되는 배열이 서로 다른 모양을 가지고 있을 때, 자동으로 호출된다.

In [71]:
arr5 = np.array([7, 8])

In [72]:
arr5.shape

(2,)

In [73]:
arr2 + arr5

ValueError: operands could not be broadcast together with shapes (3,4) (2,) 

위의 예제는, `arr5` 가 여러번 반복될지라도 `arr2`의 모양과 맞는 배수가 없기 때문에 오류가 발생한다.<br>
따라서 `arr2 + arr5` 는 계산될 수가 없다.<br>
브로드캐스팅에 대해서는 다음 웹페이지를 참고하면  된다.: https://numpy.org/doc/stable/user/basics.broadcasting.html .

###  배열 비교

넘파이 배열은 비교 연산자도 지원한다.  `==`, `!=`, `>`. 연산의 결과는 Boolean 형태로 출력된다.

In [74]:
arr1 = np.array([[1, 2, 3], [3, 4, 5]])
arr2 = np.array([[2, 2, 3], [1, 2, 5]])

In [75]:
arr1 == arr2

array([[False,  True,  True],
       [False, False,  True]])

In [76]:
arr1 != arr2

array([[ True, False, False],
       [ True,  True, False]])

In [77]:
arr1 >= arr2

array([[False,  True,  True],
       [ True,  True,  True]])

In [78]:
arr1 < arr2

array([[ True, False, False],
       [False, False, False]])

배열 비교 연산 두개의 배열에 대해 `sum` 함수를 사용하기 전에 배열간의 값을 비교하기 위해 빈번하게 사용된다.<br>
__`True`값이 `1`로 표현되고 `False`값이 `0` 으로 표현된다__는것을 명심하자

In [79]:
(arr1 == arr2).sum()

3

---
## 5.2 배열 인덱싱과 슬라이싱

넘파이는 파이썬 리스트의 인덱싱이 사용하는 `[]` 을 사용하여 직관적으로 값에 접근하게 해준다.<br>
`;`를 사용하는 슬라이싱도 배열의 원소를 접근하는데에 동일하게 사용될 수 있다~!

In [80]:
arr3 = np.array([
    [[11, 12, 13, 14], 
     [13, 14, 15, 19]], 
    
    [[15, 16, 17, 21], 
     [63, 92, 36, 18]], 
    
    [[98, 32, 81, 23],      
     [17, 18, 19.5, 43]]])

In [81]:
arr3.shape

(3, 2, 4)

In [82]:
# 원소 슬라이싱
arr3[1, 1, 2]

36.0

In [84]:
# 이중 배열의 생성
arr3[1:, 0:1, :2]

array([[[15., 16.]],

       [[98., 32.]]])

In [85]:
# 인덱싱과 슬라이싱을 복합적으로 사용
arr3[1:, 1, 3]

array([18., 43.])

In [86]:
# 인덱싱과 슬라이싱을 복합적으로 사용(2)
arr3[1:, 1, :3]

array([[63. , 92. , 36. ],
       [17. , 18. , 19.5]])

In [89]:
# 모양(3차원) 보다 적은 인덱스(1차원)를 사용할 때
arr3[1]

array([[15., 16., 17., 21.],
       [63., 92., 36., 18.]])

In [92]:
# 모양(3차원) 보다 적은 인덱스(2차원)를 사용할 때
arr3[:2, 1]

array([[13., 14., 15., 19.],
       [63., 92., 36., 18.]])

In [93]:
# 너무 많은 인덱싱을 사용한 경우
arr3[1,3,2,1]

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

표현과 결과가 처음에는 헷갈릴 수도 있다. 천천히 실험해 보면서 익숙해 지도록하자!!<br>
아래의 그림은 시각적으로 어떤과정을 통해 인덱싱과 슬라이싱이 이루어지는지 보여준다.

<img src="https://scipy-lectures.org/_images/numpy_indexing.png" width="360">

## 배열을 만드는 또다른 방법들

넘파이는 이 이외에도 고정 값이나 난수 값을 이용해서 넘파이 배열을 생성하는 방법들이 있다.<br>
다음의 웹사이트를 확인하거나 `help` 함수를 이용해서 배열 생성 방법들을 찾을 수 있다.<br>
[official documentation](https://numpy.org/doc/stable/reference/routines.array-creation.html) or use the `help` function to learn more.

In [94]:
# 원소 0으로 생성
np.zeros((3, 2))

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [95]:
# 원소 1로 생성
np.ones([2, 2, 3])

array([[[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]]])

In [96]:
# 단위 행렬 생성
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [99]:
# 랜덤 값으로 생성(실수)
np.random.rand(5)

array([0.04680116, 0.06911503, 0.59750036, 0.18262961, 0.82070366])

In [100]:
# 랜덤 값으로 생성(정수)
np.random.randn(2, 3) 

array([[0.61183741, 1.01942618, 1.45234318],
       [0.45413521, 1.32372203, 1.86886   ]])

In [101]:
# 고정 값
np.full([2, 3], 42)

array([[42, 42, 42],
       [42, 42, 42]])

In [102]:
# 시작 값과 끝 값, 그리고 증가 값들을 알려줄 때의 생성(1)
np.arange(10, 90, 3)

array([10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58,
       61, 64, 67, 70, 73, 76, 79, 82, 85, 88])

In [103]:
# 시작 값과 끝 값, 그리고 증가 값들을 알려줄 때의 생성(2)
np.linspace(3, 27, 9)

array([ 3.,  6.,  9., 12., 15., 18., 21., 24., 27.])

## 요약

이번 장을 통해서, 우리는 넘파이를 이용한 수치 연산을 알아볼 수 있었다.<br>
다음과 같은 주제들이 이번 튜토리얼을 통해 학습되었다.

- 파이썬 리스트에서 넘파이 배열로 전환
- 넘파이 배열에서의 연산
- 넘파이가 리스트에 비해 가지는 이점
- 다차원 넘파이 배열
- CSV 데이터 파일을 다루는 법
- 배열 인덱싱과 슬라이싱
- 넘파이 배열을 생성하는 여러가지 방법

넘파이에 대해 보다 많은 정보는 아래 웹페이지들을 참고 할 수 있다.

- Official tutorial: https://numpy.org/devdocs/user/quickstart.html
- Numpy tutorial on W3Schools: https://www.w3schools.com/python/numpy_intro.asp
- Advanced Numpy (exploring the internals): http://scipy-lectures.org/advanced/advanced_numpy/index.html

## 복습을 위한 질문

이 노트북에서 다루는 주제에 대한 이해도를 테스트하기 위해 다음 질문에 답해 보십시오.

1. 벡터란 무엇인가?
2. Python의 list자료형을 사용하여 벡터를 어떻게 표현합니까? 예를 들어 보세요.
3. 두 벡터의 내적(dot product)은 무엇입니까?
4. 두 벡터의 내적을 계산하는 함수를 작성하십시오.
5. Numpy가 뭔가요?
6. Numpy를 어떻게 설치합니까?
7. `numpy` 모듈을 어떻게 가져오나요?
8. 별칭(Alias)가 있는 모듈을 가져온다는 것은 무엇을 의미합니까? 예를 들어 보세요.
9. `numpy`에 일반적으로 사용되는 별칭은 무엇인지 써보세요?
10. Numpy 배열이란 무엇입니까?
11. Numpy 배열은 어떻게 생성합니까? 예를 들어 주십시오.
12. Numpy 배열의 유형은 무엇입니까?
13. Numpy 배열의 요소에 어떻게 접근하는지?
14. Numpy를 사용하여 두 벡터의 내적을 어떻게 계산합니까?
15. 크기가 다른 두 벡터의 내적을 계산하려고 하면 어떻게 됩니까?
16. 두 Numpy 배열의 요소별 곱을 어떻게 계산합니까?
17. Numpy 배열에 있는 모든 요소의 합을 어떻게 계산합니까?
18. 숫자 데이터 작업을 위해 Python 목록보다 Numpy 배열을 사용하면 어떤 장점이 있는지?
19. Numpy 배열 연산이 Python 함수 및 루프에 비해 성능이 더 나은 이유는 무엇인지?
20. 예제를 사용하여 Numpy 배열 작업과 Python 루프 간의 성능 차이를 설명해 보세요.
21. 다차원 Numpy 배열이란 무엇입니까?
22. 2, 3, 4차원의 Numpy 배열 생성을 보여보세요.
23. Numpy 배열에서 차원의 수와 각 차원의 길이를 어떻게 검사하는지 설명해보세요?
24. Numpy 배열의 요소가 다른 데이터 유형(dtype)을 가질 수 있습니까?
25. Numpy 배열 요소의 데이터 유형(dtype)을 어떻게 확인합니까?
26. Numpy 배열의 데이터 유형(data type)은 무엇입니까?
27. 행렬과 2D Numpy 배열의 차이점은 무엇입니까?
28. Numpy를 사용하여 행렬 곱셈(matrix multiplication)을 수행하는 방법은 무엇입니까?
29. Numpy에서 `@` 연산자는 무엇에 사용됩니까?
30. CSV 파일 형식은 무엇입니까?
31. Numpy를 사용하여 CSV 파일에서 데이터를 어떻게 읽습니까?
32. 두 개의 Numpy 배열을 어떻게 연결(concatenate)합니까?
33. `np.concatenate`의 `axis` 인수(argument)의 목적은 무엇입니까?
34. 두 개의 Numpy Array가 연결이 가능한 경우는 경우는 언제입니까?
35. 연결할 수 있는 두 개의 Numpy 배열의 예를 들어 보세요.
36. 연결할 수 없는 두 개의 Numpy 배열의 예를 들어 보세요.
37. `np.reshape` 함수의 목적은 무엇입니까?
38. Numpy 배열을 "재구성(reshape)"한다는 것은 무엇을 의미합니까?
39. numpy 배열을 CSV 파일에 어떻게 쓰나요?
40. 수학 연산을 수행하기 위한 Numpy 함수의 몇 가지 예를 들어 보십시오.
41. 배열 조작(array manipulation)을 수행하기 위한 Numpy 함수의 몇 가지 예를 제시하십시오.
42. 선형 대수(linear algebra)를 수행하기 위한 Numpy 함수의 몇 가지 예를 들어 보십시오.
43. 통계 연산을 수행하기 위한 Numpy 함수의 몇 가지 예를 들어보세요.
44. 특정 작업이나 사용 사례에 적합한 Numpy 기능을 어떻게 찾는지 설명해보세요?
45. 모든 Numpy 배열 함수 및 작업 목록은 어디에서 볼 수 있는지 설명해보세요?
46. ​​Numpy 배열에서 지원하는 산술 연산자는 무엇입니까? 예를 들어 설명해보세요.
47. Array BroadCasting이란? 어떻게 유용한지? 예를 들어 설명해보세요.
48. BroadCasting Array의 예를 몇 가지 들어보세요.
49. BroadCasting Array가 되지 않는 Array의 예를 몇 가지를 들어보세요.
50. Numpy 배열에서 지원하는 비교 연산자는 무엇입니까? 예를 들어 설명해 보세요.
51. Numpy 배열에서 특정 하위 배열 또는 슬라이스에 어떻게 액세스합니까?
52. 몇 가지 예를 들어 다차원 Numpy 배열에서 배열 인덱싱 및 슬라이싱을 설명해 보세요.
53. 모든 0을 포함하는 주어진 모양으로 Numpy 배열을 어떻게 생성합니까?
54. 모든 것을 포함하는 주어진 모양으로 Numpy 배열을 어떻게 생성합니까?
55. 주어진 모양의 단위 행렬을 어떻게 생성하는지 설명해보세요.
56. 주어진 길이의 랜덤 벡터를 어떻게 생성하는지 설명해보세요.
57. 각 요소에 대해 고정된 값으로 주어진 모양으로 Numpy 배열을 어떻게 생성한는지 설명해보세요.
58. 무작위로 초기화된 요소를 포함하는 주어진 모양으로 Numpy 배열을 어떻게 생성하는지 설명해보세요.
59. `np.random.rand`와 `np.random.randn`의 차이점은 무엇인지 예를 들어 설명해보세요.
60. `np.arange`와 `np.linspace`의 차이점은 무엇인지 예를 들어 설명해보세요.